# Предсказание стоимости жилья в Калифорнии

## Введение

В проекте вам нужно обучить модель линейной регрессии на данных о жилье в Калифорнии в 1990 году. На основе данных нужно предсказать медианную стоимость дома в жилом массиве. Обучите модель и сделайте предсказания на тестовой выборке. Для оценки качества модели используйте метрики RMSE, MAE и R2.

### Описание признаков

В колонках датасета содержатся следующие данные:

•	longitude — широта;

•	latitude — долгота;

•	housing_median_age — медианный возраст жителей жилого массива;

•	total_rooms — общее количество комнат в домах жилого массива;

•	total_bedrooms — общее количество спален в домах жилого массива;

•	population — количество человек, которые проживают в жилом массиве;

•	households — количество домовладений в жилом массиве;

•	median_income — медианный доход жителей жилого массива;

•	median_house_value — медианная стоимость дома в жилом массиве;

•	ocean_proximity — близость к океану.


### Импорт библиотек

Импортируем нужные библиотеки и инициализируем Spark-сессию.

In [1]:
import pandas as pd 
import numpy as np
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F

from pyspark.ml.feature import StringIndexer, VectorAssembler, StandardScaler

pyspark_version = pyspark.__version__
if int(pyspark_version[:1]) == 3:
    from pyspark.ml.feature import OneHotEncoder    
elif int(pyspark_version[:1]) == 2:
    from pyspark.ml.feature import OneHotEncodeEstimator

from pyspark.ml.regression import LinearRegression

from pyspark.ml.evaluation import RegressionEvaluator

In [2]:
spark = SparkSession.builder \
                    .master("local") \
                    .appName("California Housing Predicting House Value") \
                    .getOrCreate()

In [3]:
# зададим константы
RANDOM_SEED = 2023

### Знакомство с данными

Прочитаем данные из файла '/datasets/housing.csv' с помощью pySpark. Выведем типы данных колонок датасета.

In [4]:
df_housing = spark.read.load(
    '/datasets/housing.csv', 
    format='csv',
    sep=',',
    inferSchema=True,
    header='true'
)
df_housing.printSchema()

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)



In [5]:
# выведем названия колонок 
print(pd.DataFrame(df_housing.dtypes, columns=['column', 'type']))

               column    type
0           longitude  double
1            latitude  double
2  housing_median_age  double
3         total_rooms  double
4      total_bedrooms  double
5          population  double
6          households  double
7       median_income  double
8  median_house_value  double
9     ocean_proximity  string


In [6]:
# выведите первые 10 строк 
df_housing.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|       NEAR BAY|
|  -122.25|   37.85|              52.0|     1274.0|         235.0|     558.0|     219.0|       5.6431|          341300.0|       NEAR BAY|
|  -122.25|   37.85|              

In [7]:
# посмотрим базовые статистики
# df_housing.toPandas().describe()
for column in df_housing.columns:
    df_housing.describe(column).show()

+-------+-------------------+
|summary|          longitude|
+-------+-------------------+
|  count|              20640|
|   mean|-119.56970445736148|
| stddev|  2.003531723502584|
|    min|            -124.35|
|    max|            -114.31|
+-------+-------------------+

+-------+-----------------+
|summary|         latitude|
+-------+-----------------+
|  count|            20640|
|   mean| 35.6318614341087|
| stddev|2.135952397457101|
|    min|            32.54|
|    max|            41.95|
+-------+-----------------+

+-------+------------------+
|summary|housing_median_age|
+-------+------------------+
|  count|             20640|
|   mean|28.639486434108527|
| stddev| 12.58555761211163|
|    min|               1.0|
|    max|              52.0|
+-------+------------------+

+-------+------------------+
|summary|       total_rooms|
+-------+------------------+
|  count|             20640|
|   mean|2635.7630813953488|
| stddev|2181.6152515827944|
|    min|               2.0|
|    max|  

+-------+---------------+
|summary|ocean_proximity|
+-------+---------------+
|  count|          20640|
|   mean|           null|
| stddev|           null|
|    min|      <1H OCEAN|
|    max|     NEAR OCEAN|
+-------+---------------+



Исследуем данные на наличие пропусков.

In [8]:
# подсчет пропусков по столбцам
for column in df_housing.columns:
    print(f'{column:20} {df_housing.select(column).count() - df_housing.select(column).dropna().count()}')

longitude            0
latitude             0
housing_median_age   0
total_rooms          0
total_bedrooms       207
population           0
households           0
median_income        0
median_house_value   0
ocean_proximity      0


### Вывод

Нам доступны данные о жилье в Калифорнии в 1990 году. В них содержатся записи о 24 640 наблюдениях. 

Всего имеется 10 столбцов: 9 характеристик и целевой признак (медианная стоимость дома в жилом массиве). 

Один столбец содержит строковые данные (близость к океану), остальные – числовые. 

Пропуски есть только в столбце с общим количеством спален в домах жилого массива.

## Подготовка данных

### Создание тестового датасета

Разделим датасет на две части – тренировочную и тестовую выборку в отношении 4 : 1. Закономерности для преобразования данных (заполнения пропусков, масштабирования и т.д.) будем вырабатывать на тренировочных данных, а применять преобразования будем к обеим выборкам.

In [9]:
# разбивка на тренировочную и тестовую выборки
train_data, test_data = df_housing.randomSplit([.8,.2], seed=RANDOM_SEED)
print('train:',train_data.count(), '\ntest:', test_data.count()) 

train: 16519 
test: 4121


In [10]:
# целевой признак
target = 'median_house_value'

In [11]:
# столбец с категориальным признаком
categorical_col = 'ocean_proximity'

### Обработка пропусков

Пропуски содержатся в столбце 'total_bedrooms'. Заполним их медианным значением этого столбца.

Вычислим 50-й перцентиль столбца total_bedrooms. Это и есть медианное значение. Параметр relativeError выставим равным нулю, так как наш датасет имеет размер, позволяющий нам точно вычислить медиану.

In [12]:
# расчет медианы
median_total_bedrooms = train_data.approxQuantile('total_bedrooms', [0.5], 0)[0]
median_total_bedrooms

432.0

In [13]:
# количество пропусков до заполнения
train_data.filter(F.col('total_bedrooms').isNull()).count(), \
test_data.filter(F.col('total_bedrooms').isNull()).count(), 

(167, 40)

In [14]:
# заполняем пропуски медианным значением
train_data = train_data.fillna({'total_bedrooms': median_total_bedrooms})
test_data = test_data.fillna({'total_bedrooms': median_total_bedrooms})

In [15]:
# количество пропусков после заполнения
train_data.filter(F.col('total_bedrooms').isNull()).count(), \
test_data.filter(F.col('total_bedrooms').isNull()).count(), 

(0, 0)

### Преобразование категориальных значений

#### StringIndexer

В первую очередь трансформируем категориальный признак с помощью трансформера StringIndexer.

In [16]:
# определим StringIndexer
indexer = StringIndexer(inputCol=categorical_col, 
                        outputCol=categorical_col+'_idx',
                         handleInvalid='keep'
                       ) 

In [17]:
# обучим оценщик на тренировочной выборке
indexer = indexer.fit(train_data)

In [18]:
# трансформируем категориальный признак на тренировочной выборке
train_data = indexer.transform(train_data)
train_data.printSchema()

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = false)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)
 |-- ocean_proximity_idx: double (nullable = false)



In [19]:
# трансформируем категориальный признак на тестовой выборке
test_data = indexer.transform(test_data)
test_data.printSchema()

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = false)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)
 |-- ocean_proximity_idx: double (nullable = false)



In [20]:
# посмотрим на значения в индексированной колонке
test_data.select('ocean_proximity_idx').distinct().collect()

[Row(ocean_proximity_idx=0.0),
 Row(ocean_proximity_idx=1.0),
 Row(ocean_proximity_idx=4.0),
 Row(ocean_proximity_idx=3.0),
 Row(ocean_proximity_idx=2.0)]

#### OneHotEncoder

Преобразуем колонку с категориальными значениями техникой One hot encoding.

In [21]:
# определим OneHotEncoder
encoder = OneHotEncoder(inputCol='ocean_proximity_idx',
                        outputCol='ocean_proximity_idx_ohe')

In [22]:
# обучим оценщик на тренировочной выборке
encoder = encoder.fit(train_data)

In [23]:
# преобразуем индексированный категориальный признак на тренировочной выборке
train_data = encoder.transform(train_data)
train_data.printSchema()

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = false)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)
 |-- ocean_proximity_idx: double (nullable = false)
 |-- ocean_proximity_idx_ohe: vector (nullable = true)



In [24]:
# преобразуем индексированный категориальный признак на тестовой выборке
test_data = encoder.transform(test_data)
test_data.printSchema()

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = false)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)
 |-- ocean_proximity_idx: double (nullable = false)
 |-- ocean_proximity_idx_ohe: vector (nullable = true)



In [25]:
# посмотрим на значения внутри ocean_proximity_idx_ohe
test_data.select('ocean_proximity_idx_ohe').show(5)

+-----------------------+
|ocean_proximity_idx_ohe|
+-----------------------+
|          (5,[2],[1.0])|
|          (5,[2],[1.0])|
|          (5,[2],[1.0])|
|          (5,[2],[1.0])|
|          (5,[2],[1.0])|
+-----------------------+
only showing top 5 rows



### Генерация синтетических признаков

Создадим несколько новых столбцов с признаками:

- rooms_per_household: отношение количества комнат total_rooms к количеству домовладений households.
- population_in_household: отношение количества жителей population к количеству домовладений households. 
- bedroom_index: отношение количества спален total_bedrooms к общему количеству комнат total_rooms.


In [26]:
# для тренировочной выборки
train_data = train_data.withColumn(
    'rooms_per_household',
    F.col('total_rooms') / F.col('households')
).withColumn(
    'population_in_household',
    F.col('population') / F.col('households')
).withColumn(
    'bedroom_index',
    F.col('total_bedrooms') / F.col('total_rooms')
)

train_data.printSchema()

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = false)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)
 |-- ocean_proximity_idx: double (nullable = false)
 |-- ocean_proximity_idx_ohe: vector (nullable = true)
 |-- rooms_per_household: double (nullable = true)
 |-- population_in_household: double (nullable = true)
 |-- bedroom_index: double (nullable = true)



In [27]:
# для тестовой выборки
test_data = test_data.withColumn(
    'rooms_per_household',
    F.col('total_rooms') / F.col('households')
).withColumn(
    'population_in_household',
    F.col('population') / F.col('households')
).withColumn(
    'bedroom_index',
    F.col('total_bedrooms') / F.col('total_rooms')
)

test_data.printSchema()

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = false)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)
 |-- ocean_proximity_idx: double (nullable = false)
 |-- ocean_proximity_idx_ohe: vector (nullable = true)
 |-- rooms_per_household: double (nullable = true)
 |-- population_in_household: double (nullable = true)
 |-- bedroom_index: double (nullable = true)



Создадим переменные с названиями столбцов, необходимые для дальнейшей работы.

In [28]:
# список числовых столбцов
numerical_cols = ['longitude',
 'latitude',
 'housing_median_age',
 'total_rooms',
 'total_bedrooms',
 'population',
 'households',
 'median_income',
 'rooms_per_household',
 'population_in_household',
 'bedroom_index']

### Вектора признаков

Объединим признаки в вектора, с которым ML-алгоритм умеет работать.

В нашем наборе данных только один категориальный признак. После его обработки OneHotEncoder  вернул нам вектор, который хранится в столбце 'ocean_proximity_idx_ohe'. Поэтому дополнительно обрабатывать этот столбец с помощью VectorAssembler нет смысла.

In [29]:
# ассемблер вектора числовых признаков
numerical_assembler = VectorAssembler(
    inputCols=numerical_cols,
    outputCol="numerical_features"
)

In [30]:
# добавляем вектор числовых признаков к тренировочным данным
train_data = numerical_assembler.transform(train_data) 

In [31]:
# посмотрим пример получившегося вектора
train_data.select('numerical_features').show(5, truncate=False)

+--------------------------------------------------------------------------------------------------------------+
|numerical_features                                                                                            |
+--------------------------------------------------------------------------------------------------------------+
|[-124.35,40.54,52.0,1820.0,300.0,806.0,270.0,3.0147,6.7407407407407405,2.9851851851851854,0.16483516483516483]|
|[-124.3,41.8,19.0,2672.0,552.0,1298.0,478.0,1.9797,5.589958158995816,2.715481171548117,0.20658682634730538]   |
|[-124.3,41.84,17.0,2677.0,531.0,1244.0,456.0,3.0313,5.870614035087719,2.7280701754385963,0.19835636906985432] |
|[-124.27,40.69,36.0,2349.0,528.0,1194.0,465.0,2.5179,5.051612903225807,2.567741935483871,0.2247765006385696]  |
|[-124.26,40.58,52.0,2217.0,394.0,907.0,369.0,2.3571,6.008130081300813,2.4579945799457996,0.17771763644564728] |
+-----------------------------------------------------------------------------------------------

In [32]:
# добавляем вектор числовых признаков к тестовым данным
test_data = numerical_assembler.transform(test_data) 

In [33]:
# проверим столбцы
test_data.printSchema()

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = false)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)
 |-- ocean_proximity_idx: double (nullable = false)
 |-- ocean_proximity_idx_ohe: vector (nullable = true)
 |-- rooms_per_household: double (nullable = true)
 |-- population_in_household: double (nullable = true)
 |-- bedroom_index: double (nullable = true)
 |-- numerical_features: vector (nullable = true)



### Шкалирование значений

Для числовых признаков тоже нужна трансформация — шкалирование значений — чтобы сильные выбросы не смещали предсказания модели.

In [34]:
# инициализируем StandardScaler
standardScaler = StandardScaler(
    inputCol='numerical_features',
    outputCol="numerical_features_scaled"
)

In [35]:
# обучаем StandardScaler на тренировочной выборке
standardScaler = standardScaler.fit(train_data)

In [36]:
# масштабируем числовые признаки тренировочной выборки
train_data = standardScaler.transform(train_data)

In [37]:
# посмотрим пример получившегося вектора
train_data.select('numerical_features_scaled').show(5, truncate=False)

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|numerical_features_scaled                                                                                                                                                                                         |
+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[-62.02964256566471,18.99375002446522,4.136996929296689,0.8514654800992952,0.7273133614390778,0.7427055124812206,0.7186026186759754,1.5839557840149008,2.681443914416072,0.2573081351742053,2.4884194570386304]   |
|[-62.004701012562315,19.58408364634056,1.511595031858406,1.2500636059479764,1.338256585047903,1.1960691751868788,1.2721927841745047,1.0401556591416

In [38]:
# масштабируем числовые признаки тестовой выборки
test_data = standardScaler.transform(test_data)

In [39]:
# проверим столбцы
test_data.printSchema()

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = false)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)
 |-- ocean_proximity_idx: double (nullable = false)
 |-- ocean_proximity_idx_ohe: vector (nullable = true)
 |-- rooms_per_household: double (nullable = true)
 |-- population_in_household: double (nullable = true)
 |-- bedroom_index: double (nullable = true)
 |-- numerical_features: vector (nullable = true)
 |-- numerical_features_scaled: vector (nullable = true)



### Вывод

На этапе подготовки данных мы разделили датасет на две части – тренировочную и тестовую выборку в отношении 4 : 1.

Пропуски содержались только в столбце total_bedrooms с общим количеством спален в домах жилого массива. Заполнили их медианным значением по столбцу.

Столбец ocean_proximity содержит категориальный признак - близость к океану. Обработали этот признак следующим образом: сначала привели его числовому виду с помощью StringIndexer, затем преобразовали с помощью техники One hot encoding.

Для повышения качества модели создали несколько новых столбцов с признаками:
    
- rooms_per_household: отношение количества комнат total_rooms к количеству домовладений households.
- population_in_household: отношение количества жителей population к количеству домовладений households.
- bedroom_index: отношение количества спален total_bedrooms к общему количеству комнат total_rooms. 

Далее числовые признаки собрали в вектор, который затем масштабировали с помощью StandardScaler.
Теперь можно переходить к обучению модели.


## Обучение моделей

Построим две модели линейной регрессии на разных наборах данных: 

- используя все данные из файла;

- используя только числовые переменные, исключив категориальные.

### Наборы данных для обучения

Соберем трансформированные категорийные и числовые признаки с помощью VectorAssembler.

In [40]:
# для обучения модели по всем данным
all_features = ['ocean_proximity_idx_ohe','numerical_features_scaled']

In [41]:
# ассемблер всех признаков
all_features_assembler = VectorAssembler(
    inputCols=all_features, 
    outputCol="all_features")

In [42]:
# собираем все признаки для тренировочной выборки
train_data = all_features_assembler.transform(train_data)

In [43]:
# посмотрим пример получившегося вектора
train_data.select('all_features').show(1, truncate=False)

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|all_features                                                                                                                                                                                                                       |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[0.0,0.0,1.0,0.0,0.0,-62.02964256566471,18.99375002446522,4.136996929296689,0.8514654800992952,0.7273133614390778,0.7427055124812206,0.7186026186759754,1.5839557840149008,2.681443914416072,0.2573081351742053,2.4884194570386304]|
+-------------------------------------------------------------------------------

In [44]:
# собираем все признаки для тестовой выборки
test_data = all_features_assembler.transform(test_data)

In [45]:
# проверим столбцы
test_data.printSchema()

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = false)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)
 |-- ocean_proximity_idx: double (nullable = false)
 |-- ocean_proximity_idx_ohe: vector (nullable = true)
 |-- rooms_per_household: double (nullable = true)
 |-- population_in_household: double (nullable = true)
 |-- bedroom_index: double (nullable = true)
 |-- numerical_features: vector (nullable = true)
 |-- numerical_features_scaled: vector (nullable = true)
 |-- all_features: vector (nullable = true)



Для построения модели будем использовать оценщик LinearRegression из библиотеки MLlib.

### Регрессия по всем данным

In [46]:
# инициализируем линейную регрессию
lr_all = LinearRegression(labelCol=target, featuresCol='all_features')

In [47]:
# обучаем модель на тренировочной выборке
lr_all = lr_all.fit(train_data)

23/07/14 17:08:27 WARN Instrumentation: [c70d7b66] regParam is zero, which might cause numerical instability and overfitting.
23/07/14 17:08:27 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
23/07/14 17:08:27 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
23/07/14 17:08:28 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
23/07/14 17:08:28 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK


In [48]:
# создаем предсказания для тестовой выборки
predictions_on_all = lr_all.transform(test_data)

In [49]:
predictions_on_all.select('median_house_value', 'prediction').show(10)

+------------------+------------------+
|median_house_value|        prediction|
+------------------+------------------+
|           78300.0|122851.96173983812|
|           67000.0| 149528.3170852661|
|           62500.0|163295.19871732593|
|          100600.0| 188242.8727504313|
|          104200.0|195876.33860859275|
|           74100.0|154340.64693838358|
|          128100.0|216722.72860649228|
|          130600.0|214340.63430464268|
|           92800.0|207301.13498076797|
|           83000.0| 173963.9000453055|
+------------------+------------------+
only showing top 10 rows



### Регрессия по числовым характеристикам

In [50]:
# инициализируем линейную регрессию
lr_num = LinearRegression(labelCol=target, featuresCol='numerical_features_scaled')

In [51]:
# обучаем модель на тренировочной выборке
lr_num = lr_num.fit(train_data)

23/07/14 17:08:31 WARN Instrumentation: [736c9be9] regParam is zero, which might cause numerical instability and overfitting.


In [52]:
# создаем предсказания для тестовой выборки
predictions_on_num = lr_num.transform(test_data)

In [53]:
predictions_on_num.select('median_house_value', 'prediction').show(10)

+------------------+------------------+
|median_house_value|        prediction|
+------------------+------------------+
|           78300.0| 78126.04342938773|
|           67000.0|119507.87554582488|
|           62500.0|133889.68745061755|
|          100600.0|157414.14855087502|
|          104200.0|165685.66417634767|
|           74100.0|124235.44595741061|
|          128100.0|189241.86645224597|
|          130600.0|186014.30771065038|
|           92800.0| 176969.7893705424|
|           83000.0|142409.18660525745|
+------------------+------------------+
only showing top 10 rows



Предсказания модели, которая училась только на числовых признаках выглядят более близкими к целевому признаку. Получим более точную оценку ошибки, рассчитав несколько метрик для двух моделей.

### Вывод

На этапе обучения модели мы создали дополнительный столбец all_features, который представляет собой вектор в пространстве масштабированных числовых признаков и обработанного техникой One hot encoding категориального признака.

Модель линейной регрессии обучалась два раза: на всех имеющихся характеристиках и только на числовых.

Перейдем к оценке качества прогнозов полученной модели.

## Анализ результатов

Сравним результаты работы линейной регрессии на двух наборах данных по метрикам RMSE, MAE и R2.

### Метрика RMSE

RMSE (Root Mean Squared Error), это корень квадратный из среднеквадратичной ошибки прогноза. Эта метрика показывает, на сколько, в среднем, прогноз отклоняется от реальных значений рыночной цены.

In [55]:
# величина ошибки на всех признаках
rmse_all = RegressionEvaluator(
    labelCol='median_house_value',
).evaluate(predictions_on_all)

rmse_all

69278.94079842806

In [56]:
# величина ошибки на числовых признаках
rmse_num = RegressionEvaluator(
    labelCol='median_house_value',
    metricName='rmse'
).evaluate(predictions_on_num)

rmse_num
# 70610.93156332383

70260.62561297978

### Метрика MAE

Средняя абсолютная ошибка (MAE). Это степень несоответствия между фактическими и прогнозируемыми значениями. Абсолютная ошибка представляет собой разность между спрогнозированным и фактическим значениями. MAE — это среднее от таких ошибок, что помогает понять эффективность модели.

In [58]:
# величина ошибки на всех признаках
mae_all = RegressionEvaluator(
    labelCol='median_house_value',
    metricName='mae'
).evaluate(predictions_on_all)

mae_all

49359.56970419781

In [59]:
# величина ошибки на числовых признаках
mae_num = RegressionEvaluator(
    labelCol='median_house_value',
    metricName='mae'
).evaluate(predictions_on_num)
mae_num

50372.631036849154

### Метрика R2

R2 - это показатель хорошей подгонки модели. В регрессии коэффициент детерминации R2 является статистической мерой того, насколько хорошо предсказания регрессии приближаются к реальным точкам данных. Значение R2, равное 1, указывает на то, что предсказания регрессии идеально соответствуют данным.

In [61]:
# величина ошибки на всех признаках
r2_all = RegressionEvaluator(
    labelCol='median_house_value',
    metricName='r2'
).evaluate(predictions_on_all)

r2_all

0.6457435162598251

In [62]:
# величина ошибки на числовых признаках
r2_num = RegressionEvaluator(
    labelCol='median_house_value',
    metricName='r2'
).evaluate(predictions_on_num)
r2_num

0.6356327336797164

### Вывод

Анализ метрик моделей показал, что результаты модели, которая обучалась на всех данных выше результатов модели, обученной только на числовых данных. Прогноз отклоняется от реальных значений в среднем на 69 278 $.

Величина R2 в 0,65 говорит, что модель недостаточно хорошо подогнана к обучающим данным.

Возможные дальнейшие пути улучшения качества прогнозов:

- выбор более мощной модели
- генерация большего числа новых признаков
- более тонкая настройка гиперпараметров модели.


## Общий вывод

В данном проекте нужно **предсказать медианную стоимость дома в жилом массиве** на данных о жилье в Калифорнии в 1990 году. На основе данных нужно обучить модель линейной регрессии с помощью библиотек pyspark и модуля MLlib. 

Для обучения модели линейной регрессии нам доступны **данные о жилье в Калифорнии в 1990 году**. В них содержатся записи о 24 640 наблюдениях.

Всего имеется 10 столбцов: 9 характеристик и целевой признак (медианная стоимость дома в жилом массиве). Один столбец содержит строковые данные (близость к океану), остальные – числовые.

На этапе подготовки данных мы разделили датасет на две части – тренировочную и тестовую выборку в отношении 4 : 1.

**Пропуски** содержались только в столбце total_bedrooms с общим количеством спален в домах жилого массива. Заполнили их медианным значением по столбцу.

**Категориальный признак обработали** следующим образом: сначала привели его числовому виду с помощью StringIndexer, затем преобразовали с помощью техники One hot encoding. 

Для повышения качества модели **создали несколько** новых **столбцов с числовыми признаками** (rooms_per_household, population_in_household и bedroom_index). 

Числовые признаки масштабировали с помощью StandardScaler.

**Модель линейной регрессии обучили два раза**: на всех имеющихся характеристиках (lr_all) и только на числовых (lr_num).

Для оценки качества получившихся моделей сравнили результаты работы линейной регрессии на двух наборах данных по метрикам **RMSE, MAE и R2**.

Анализ метрик моделей показал, что **результаты модели, которая обучалась на всех данных выше результатов модели, обученной только на числовых данных**. Прогноз отклоняется от реальных значений в среднем на 69 278 $.

Величина R2 в 0,65 говорит, что модель недостаточно хорошо подогнана к обучающим данным.

Возможные **дальнейшие пути улучшения качества прогнозов**:

- выбор более мощной модели
- генерация большего числа новых признаков
- более тонкая настройка гиперпараметров модели.


In [63]:
spark.stop()